<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/wordmark_black65ee464.png" width="700">

# Week 2 : Final Challenge

**Welcome to the final challenge!**  

In the previous notebook we've seen how we can use the VQC class in Aqua to classify the digits `0` and `1`. However, classifying `0` and `1` is relatively simple as digits `0` and `1` are easily distinguishable. `4` and `9` however, are notoriously similar, with a _loop_ on the top and a _line_ on the bottom. This can be corroborated looking at our 2-D t-SNE plot from the previous notebook (Fig.2), we see that `0` and `1` are clustered relatively far from each other making them easily distinguishable, however `4` and `9` are overlapping. In this challenge we are providing you with a dataset with digits reduced to **dimension 3**. For example, in Fig.1 we can see the dimension reduction of the 784 dimension vector for digit `4` into a dimension 3 feature vector. 

**Fig.1 : Features of the digit `4` after reducing dimension to 3:** 
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/four2a7701f.png" width="700">

**Fig.2 : MNIST dataset after dimension reduction to 2 as given in the previous notebook:**
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/mnist_plot53adb39.png" width="400">

## Challenge Question   
Use the VQC method from Aqua to classify the digits `4` and `9` as given in the dataset **challenge_dataset_4_9_dim3_training.csv** provided to you. 

## Rules and Guidelines

* Your `QuantumCircuit` can have a **maximum of 6 qubits**.
* **Cost of the circuit should be less than 2000**.  
* You should not change names of the functions `feature_map()` , `variational_circuit()`  and `return_optimal_params()`.
* All the functions must return the value types as mentioned. 
* All circuits must be Qiskit generated.
* Best of all submissions is considered for grading.

## Judging criteria 

* Primary judgement is based on the **accuracy of the model**, higher the better. **Accuracies which differ by less than 0.005 will be considered to be equal**. ex: Accuracies 0.7783 and 0.7741 will be considered to be equal.
* If the accuracies are tied, the tie will be broken using **cost of the circuit** as the metric, lower the better. 
* In the case that both accuracy of the model and cost of the circuit are equal, **time of submission** is taken into account, Earlier the better. 

_**Important Note:**_ The **leaderboard shown during the progress of the competition** will only display accuracy of the model and is **not the final leaderboard**. Breaking ties between accuracy of the model by considering lower **cost of circuit** will only be done after the competition ends. **The final leaderboard will be announced post the event** which will take into consideration cost of the circuit and time of submission. 

## Certificate Eligibility

Everyone who scores an **accuracy greater than 0.70 (i.e, 70%) will be eligible for a certificate**. 


An explanation on how to calculate the accuracy of the model and the cost of the circuit is given in the end inside the `grade()` function. Before you submit, make sure the grading function is running on your device. To save time you can also use the grading function provided to calculate the accuracy and circuit cost without having to submit your solution onto HackerEarth. Remember, your final score will be determined using the same grading methods as given in this notebook, but will be evaluated on unseen datapoints.

In [1]:
# installing a few dependencies
!pip install --upgrade seaborn==0.10.1
!pip install --upgrade scikit-learn==0.23.1
!pip install --upgrade matplotlib==3.2.0
!pip install --upgrade pandas==1.0.4
!pip install --upgrade qiskit==0.19.6 
!pip install --upgrade plotly==4.9.0

# the output will be cleared after installation
from IPython.display import clear_output
clear_output()

In [24]:
# we have imported a few libraries we thing might be useful 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer

from qiskit import *
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
%matplotlib inline
import matplotlib.pyplot as plt

import time
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, RealAmplitudes, EfficientSU2
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import COBYLA


# The the write_and_run() magic function creates a file with the content inside the cell that it is run. 
# You have used this in previous exercises for creating your submission files. 
# It will be used for the same purpose here.

from IPython.core.magic import register_cell_magic
@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

# Solution

## Data loading 

This notebook has helper functions and code snippets to save your time and help you concentrate on what's important: Increasing the accuracy of your model. Running the cell below will import the challenge dataset and will be available to you as `data`. Before running the cell below store the dataset in this file structure (or change the `data_path` accordingly):  

- `challenge_notebook.ipynb`
- `dataset`
    - `challenge_dataset_4_9.csv`


In [25]:
data_path='./dataset/'
data = np.loadtxt(data_path + "challenge_dataset_4_9_dim3_training.csv", delimiter=",")

# extracting the first column which contains the labels
data_labels = data[:, :1].reshape(data.shape[0],)
# extracting all the columns but the first which are our features
data_features = data[:, 1:]

## Visualizing the dataset

Before we dive into solving the question it is always beneficial to look at the dataset pictographically. This will help us understand patterns which we could leverage when designing our feature maps and variational circuits for example.

In [26]:
import plotly.express as px
import pandas as pd

# creating a dataframe using pandas only for the purpose fo plotting
df = pd.DataFrame({'Component 0':data_features[:,0], 'Component 1':data_features[:,1], 
                   'Component 2':data_features[:,2], 'label':data_labels})

fig = px.scatter_3d(df, x='Component 0', y='Component 1', z='Component 2', color='label')
fig.show()

## Extracting the training dataset

The given dataset has already been reduced in dimension and normalized, so, further pre-processing isn't techincally required. You can do so if you want to, but the testing dataset will be of the same dimension and normalisation as the training dataset provided. Training a dataset of size 6,000 will take multiple hours so you'll need to extract a subset of the dataset to use as a training dataset. The accuracy of the model may vary based on the datapoints and size of the training dataset you choose. Thus, experimenting with various sizes and datapoints will be necessary. For example, Increasing the training dataset size may increase the accuracy of the model however it will increase the training time as well.

Use the space below to extract your training dataset from `data`. For your convenience `data` has been segregated into `data_labels` and `data_features`.

* `data_labels` : 6,000 $\times$ 1 column vector with each entry either `4` or `9` 
* `data_features` : 6,000 $\times$ 3 matrix with each row having the feature corresponding to the label in `data_labels`

**Note:** This process was done in the previous [VQC notebook](https://github.com/Qiskit-Challenge-India/2020/blob/master/Day%206%2C%207%2C8/VQC_notebook.ipynb) with `0` and `1` labels and can be modified and used here as well. 

In [27]:
### WRITE YOUR CODE BETWEEN THESE LINES - START

# do your classical pre-processing here
four_datapoints = []
nine_datapoints = []
for i in range(6000):
    if data_labels[i]==4:
        four_datapoints.append(data_features[i])
for i in range(6000):
    if data_labels[i]==9:
        nine_datapoints.append(data_features[i])
four_datapoints = np.array(four_datapoints)
nine_datapoints = np.array(nine_datapoints)
four_datapoints_normalized = four_datapoints
nine_datapoints_normalized = nine_datapoints
# store your training and testing datasets to be input in the VQC optimizer in the "training_input" and 
# "testing_input" variables respectively. These variables will eb accessed whiile creating a VQC instance later. 
training_input = None
testing_input = None

train_size = 70
test_size = 30
dp_size_four = 5
dp_size_nine = 5
four_train = four_datapoints_normalized[:train_size]
nine_train = nine_datapoints_normalized[:train_size]
four_test = four_datapoints_normalized[train_size + 1:train_size + test_size + 1]
nine_test = nine_datapoints_normalized[train_size + 1:train_size + test_size + 1]
training_input = {'A':four_train, 'B':nine_train}
testing_input = {'A':four_test, 'B':nine_test}

### WRITE YOUR CODE BETWEEN THESE LINES - END

## Building a Quantum Feature Map

Given below is the `feature_map()` function. It takes no input and has to return a feature map which is either a `FeatureMap` or `QuantumCircuit` object. In the previous notebook you've learnt how feature maps work and the process of using existing feature maps in Qiskit or creating your own. In the space given **inside the function** you have to create a feature map and return it.   


**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `feature_map.py`

In [28]:
%%write_and_run feature_map.py
# the write_and_run function writes the content in this cell into the file "feature_map.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap
import numpy as np
import time
import functools
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def feature_map(): 
    # BUILD FEATURE MAP HERE - START
    
    # import required qiskit libraries if additional libraries are required
    def custom_map(x):
        coeff = 0
        if len(x)==1:
            coeff = x[0]
        elif len(x)==3:
            coeff = functools.reduce(lambda l,m,n:1*m*n,np.pi - x)
        return coeff
    
    # build the feature map
    feature_map = None

    feature_map = PauliFeatureMap(feature_dimension=3,
                                reps=5,
                                entanglement="full",
                                paulis=['Z','ZZ'],
                                data_map_func=custom_map)
    
    # BUILD FEATURE MAP HERE - END
    
    #return the feature map which is either a FeatureMap or QuantumCircuit object
    return feature_map

## Building a Variational Circuit

Given below is the `variational_circuit()` function. It takes no input and has to return a variational circuit which is either a `VariationalForm` or `QuantumCircuit` object. In the previous notebook you've learnt how variational circuits work and the process of using existing variational circuit in Qiskit or creating your own. You have to create a variational circuit in the space given **inside the function** and return it. You can find various variational circuits in the [Qiskit Circuit Library](https://qiskit.org/documentation/apidoc/circuit_library.html) under N-local circuits.

**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `variational_circuit.py`

In [29]:
%%write_and_run variational_circuit.py
# the write_and_run function writes the content in this cell into the file "variational_circuit.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import *
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def variational_circuit():
    # BUILD VARIATIONAL CIRCUIT HERE - START
    
    # import required qiskit libraries if additional libraries are required
    
    # build the variational circuit
    var_circuit = None
    
    feature_dim = 3
    var_circuit = RealAmplitudes(feature_dim, entanglement='full', reps=3)

    # BUILD VARIATIONAL CIRCUIT HERE - END
    
    # return the variational circuit which is either a VaritionalForm or QuantumCircuit object
    return var_circuit

## Choosing a Classical Optimizer

In the `classical_optimizer()` function given below you will have to import the optimizer of your choice from [`qiskit.aqua.optimizers`](https://qiskit.org/documentation/apidoc/qiskit.aqua.components.optimizers.html) and return it. This function will not be called by the grading function `grade()` and thus the name of the function `classical_optimizer()`can be changed if needed. 

In [30]:
def classical_optimizer():
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - START
    
    # import the required clasical optimizer from qiskit.aqua.optimizers
    
    # create an optimizer object
    cls_opt = None
    cls_opt = COBYLA()
    
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - END
    return cls_opt

### Callback Function

The `VQC` class can take in a callback function to which the following parameters will be passed after every optimization cycle of the algorithm:

* `eval_count` : the evaulation counter
* `var_params` : value of parameters of the variational circuit
* `eval_val`  : current cross entropy cost 
* `index` : the batch index

In [31]:
def call_back_vqc(eval_count, var_params, eval_val, index):
    print("eval_count: {}".format(eval_count))
    print("var_params: {}".format(var_params))
    print("eval_val: {}".format(eval_val))
    print("index: {}".format(index))

## Optimization Step

This is where the whole VQC algorithm will come together. First we create an instance of the `VQC` class. 

In [32]:
# a fixed seed so that we get the same answer when the same input is given. 
seed = 10598

# setting our backend to qasm_simulator with the "statevector" method on. This particular setup is given as it was 
# found to perform better than most. Feel free to play around with different backend options.
backend = Aer.get_backend('statevector_simulator')
backend_options = {"method": "statevector"}

# creating a quantum instance using the backend and backend options taken before
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed, 
                                   backend_options=backend_options)

# creating a VQC instance which you will be used for training. Make sure you input the correct training_dataset and 
# testing_dataset as defined in your program.
vqc = VQC(optimizer=classical_optimizer(), 
          feature_map=feature_map(), 
          var_form=variational_circuit(), 
          callback=call_back_vqc, 
          training_dataset=training_input,     # training_input must be initialized with your training dataset
          test_dataset=testing_input)          # testing_input must be initialized with your testing dataset

Now, let's run the VQC classification routine

In [33]:
start = time.process_time()

result = vqc.run(quantum_instance)

print("time taken: ")
print(time.process_time() - start)

print("testing success ratio: {}".format(result['testing_accuracy']))

eval_count: 0
var_params: [-0.74819103  0.8917253   1.63629119  0.58140568  1.27305226 -0.44008163
  0.2208818   1.15966647 -0.13829344 -0.42491469  0.01896251 -0.64683019]
eval_val: 1.0869168258064796
index: 0
eval_count: 1
var_params: [ 0.25180897  0.8917253   1.63629119  0.58140568  1.27305226 -0.44008163
  0.2208818   1.15966647 -0.13829344 -0.42491469  0.01896251 -0.64683019]
eval_val: 0.9840983156763384
index: 1
eval_count: 2
var_params: [ 0.25180897  1.8917253   1.63629119  0.58140568  1.27305226 -0.44008163
  0.2208818   1.15966647 -0.13829344 -0.42491469  0.01896251 -0.64683019]
eval_val: 0.8561524524833544
index: 2
eval_count: 3
var_params: [ 0.25180897  1.8917253   2.63629119  0.58140568  1.27305226 -0.44008163
  0.2208818   1.15966647 -0.13829344 -0.42491469  0.01896251 -0.64683019]
eval_val: 0.8223161885062008
index: 3
eval_count: 4
var_params: [ 0.25180897  1.8917253   2.63629119  1.58140568  1.27305226 -0.44008163
  0.2208818   1.15966647 -0.13829344 -0.42491469  0.01896

eval_count: 39
var_params: [-0.33240013  0.85425298  2.26831856 -0.77959523  3.20714635 -0.3473011
  0.65873554  1.04584557  0.10752348  0.75799407 -0.08692914 -1.32643797]
eval_val: 0.5259515699186023
index: 39
eval_count: 40
var_params: [-0.33418488  0.83990341  2.27349116 -0.63110076  3.19332763 -0.32591277
  0.61236541  1.10534462  0.11276308  0.77525567 -0.08891052 -1.54818209]
eval_val: 0.5247624126503166
index: 40
eval_count: 41
var_params: [-0.12684796  0.71609642  2.27150669 -0.65425451  3.09401279 -0.27951632
  0.6095178   1.06770198  0.13505532  0.71624816 -0.10384887 -1.49186983]
eval_val: 0.526407791961892
index: 41
eval_count: 42
var_params: [-0.24689728  0.92321346  2.29226066 -0.66325579  3.16913387 -0.31970971
  0.59915478  1.17422897  0.14723137  0.71031963 -0.06680936 -1.48599314]
eval_val: 0.523531935030127
index: 42
eval_count: 43
var_params: [-0.22713585  0.91345926  2.41911899 -0.56590648  3.24852057 -0.22839018
  0.57775454  1.06156905  0.14789264  0.84604418 -0

eval_count: 77
var_params: [-0.20558883  0.92216496  2.18949939 -0.86778072  2.86977974 -0.18989844
  0.72007114  1.0701218   0.07338289  0.87622454 -0.06561547 -1.40430033]
eval_val: 0.5159003822815165
index: 77
eval_count: 78
var_params: [-0.18568936  0.88167643  2.19114802 -0.88205628  2.8692789  -0.18643781
  0.7035587   1.05550146  0.07423684  0.85382059 -0.04031593 -1.40910193]
eval_val: 0.5171466076234065
index: 78
eval_count: 79
var_params: [-0.19636893  0.92375958  2.18460987 -0.85143672  2.85678675 -0.19810682
  0.7213703   1.0545265   0.07725102  0.87428448 -0.06810179 -1.39470997]
eval_val: 0.5160245860707974
index: 79
eval_count: 80
var_params: [-0.19920355  0.94436079  2.18124825 -0.87330231  2.87221998 -0.21062041
  0.74026756  1.0888144   0.0941634   0.91133998 -0.07409288 -1.42286914]
eval_val: 0.5161923139249287
index: 80
eval_count: 81
var_params: [-0.21427567  0.91641208  2.18968921 -0.85698733  2.87346206 -0.19377163
  0.70692769  1.07904414  0.08186774  0.8872635 

eval_count: 116
var_params: [-0.23845838  1.02774942  2.18004397 -0.91499211  2.80802857 -0.19951177
  0.67194485  1.13873607  0.07931499  0.85268182 -0.12323693 -1.42679645]
eval_val: 0.5148788129920637
index: 116
eval_count: 117
var_params: [-0.22757209  1.0146841   2.18469    -0.92118766  2.81573572 -0.19431363
  0.68348253  1.14542785  0.08469259  0.8522025  -0.12838748 -1.40331191]
eval_val: 0.5148849754032093
index: 117
eval_count: 118
var_params: [-0.24194066  1.01457142  2.18504692 -0.93571726  2.79875639 -0.20426013
  0.6922103   1.14124651  0.08238205  0.86537431 -0.14652852 -1.40997353]
eval_val: 0.5147536797962147
index: 118
eval_count: 119
var_params: [-0.24285685  1.02160667  2.17703564 -0.94781231  2.78158581 -0.1899733
  0.70418022  1.14542261  0.08617745  0.86946063 -0.14220642 -1.41248605]
eval_val: 0.5147071952948872
index: 119
eval_count: 120
var_params: [-0.24590332  1.01926627  2.16413589 -0.9606029   2.7809386  -0.19841752
  0.6986821   1.13869814  0.10764701  0.

eval_count: 155
var_params: [-0.19505936  1.06325672  2.18074911 -1.01328066  2.66955368 -0.19597882
  0.66180453  1.22751036  0.14971682  0.95879537 -0.19186807 -1.39881964]
eval_val: 0.5135814725572012
index: 155
eval_count: 156
var_params: [-0.19048742  1.06391487  2.18224987 -1.01536483  2.66702417 -0.1897089
  0.65862202  1.23080904  0.15590534  0.95527582 -0.18209065 -1.4006101 ]
eval_val: 0.5135747286591262
index: 156
eval_count: 157
var_params: [-0.19580421  1.06036071  2.16175711 -1.01425473  2.66151833 -0.18197137
  0.64619925  1.24487362  0.1506433   0.95274984 -0.18711212 -1.39702251]
eval_val: 0.5134450108124428
index: 157
eval_count: 158
var_params: [-0.21249081  1.06879944  2.15596079 -1.02958947  2.6634776  -0.18046433
  0.65297566  1.2543529   0.14355378  0.95930561 -0.17603231 -1.39754024]
eval_val: 0.5134316011703833
index: 158
eval_count: 159
var_params: [-0.20665819  1.0755466   2.15282118 -1.03593431  2.65611663 -0.18050723
  0.65590982  1.25084082  0.1406917   0.

...

eval_count: 815
var_params: [ 0.11815956  1.69595771  2.13717785 -1.15568319  1.91603931  0.11356731
  0.23606224  1.76738778  0.63220938  1.00428692 -0.4306383  -0.80131477]
eval_val: 0.5041926690650379
index: 815
eval_count: 816
var_params: [ 0.1238423   1.6978162   2.14051667 -1.1537367   1.91419595  0.11090338
  0.23425572  1.77064398  0.63626766  1.00430502 -0.42589555 -0.79840993]
eval_val: 0.5041520491823767
index: 816
eval_count: 817
var_params: [ 0.12618829  1.69867743  2.14373475 -1.15250962  1.90941283  0.11139541
  0.2330402   1.77101424  0.63331075  1.00735848 -0.42618024 -0.79819741]
eval_val: 0.5041434458618366
index: 817
eval_count: 818
var_params: [ 0.12681714  1.69761059  2.14390575 -1.15102991  1.91090513  0.11151339
  0.23199038  1.76855367  0.63282845  1.00818326 -0.42523489 -0.79764747]
eval_val: 0.5041445272689905
index: 818
eval_count: 819
var_params: [ 0.13029868  1.69889652  2.14437067 -1.15518777  1.9068938   0.10865643
  0.23360512  1.76822571  0.63591776  1

eval_count: 854
var_params: [ 0.15624099  1.72254987  2.14129993 -1.13529578  1.90264675  0.10201857
  0.22240334  1.7672503   0.62865941  0.99926039 -0.45006642 -0.7762683 ]
eval_val: 0.5039906185285778
index: 854
eval_count: 855
var_params: [ 0.16048894  1.7254417   2.14621712 -1.13404068  1.90387229  0.10095788
  0.22150545  1.76913353  0.63289484  0.99832462 -0.44659888 -0.77465492]
eval_val: 0.5039559611583495
index: 855
eval_count: 856
var_params: [ 0.16500235  1.7237489   2.14728008 -1.13327106  1.90084179  0.09785993
  0.22210462  1.76725012  0.63209249  0.99696074 -0.44941299 -0.77293273]
eval_val: 0.5039389379974677
index: 856
eval_count: 857
var_params: [ 0.16550252  1.72420277  2.14527333 -1.13342821  1.89991594  0.09674028
  0.21991675  1.76807965  0.63133961  0.99667705 -0.44787798 -0.77253072]
eval_val: 0.5039379568540404
index: 857
eval_count: 858
var_params: [ 0.16819919  1.72782762  2.14915088 -1.13142437  1.89723991  0.09771007
  0.21939611  1.76798663  0.62892017  0

eval_count: 893
var_params: [ 0.1965259   1.72933888  2.15445886 -1.11489843  1.90027911  0.08701537
  0.20971832  1.76802942  0.64395105  0.98480572 -0.46577088 -0.7459439 ]
eval_val: 0.5037801412143732
index: 893
eval_count: 894
var_params: [ 0.19695799  1.7320122   2.15880106 -1.11447396  1.89845185  0.08914534
  0.20934278  1.7667503   0.64112456  0.98882063 -0.46657247 -0.74543094]
eval_val: 0.5037761640321341
index: 894
eval_count: 895
var_params: [ 0.19577328  1.73213316  2.15897315 -1.1152682   1.89925022  0.08824971
  0.2087034   1.76701253  0.64201914  0.99015    -0.46852487 -0.74764085]
eval_val: 0.503780474436641
index: 895
eval_count: 896
var_params: [ 0.20267715  1.73534321  2.1583943  -1.11379531  1.89647962  0.08822203
  0.20831855  1.76445572  0.64315001  0.98804261 -0.46713491 -0.74470685]
eval_val: 0.5037664916487525
index: 896
eval_count: 897
var_params: [ 0.20528445  1.73443641  2.16270371 -1.11295162  1.89492355  0.0870905
  0.20504371  1.76556101  0.64268061  0.9

eval_count: 932
var_params: [ 0.2707857   1.76885967  2.17222324 -1.07456765  1.89825947  0.08392914
  0.18920145  1.77194572  0.64680421  0.96123607 -0.50362748 -0.68803957]
eval_val: 0.5034276917584326
index: 932
eval_count: 933
var_params: [ 0.27671504  1.77064716  2.17364197 -1.07288398  1.89594036  0.08201359
  0.18826519  1.77131509  0.6487838   0.96019263 -0.50298145 -0.68659433]
eval_val: 0.5034074182981332
index: 933
eval_count: 934
var_params: [ 0.27900513  1.77283998  2.17829682 -1.07115684  1.89640676  0.08181214
  0.18694002  1.77133449  0.65187363  0.96196638 -0.50570945 -0.68863927]
eval_val: 0.5034015900025253
index: 934
eval_count: 935
var_params: [ 0.28019432  1.77536079  2.17654065 -1.06802371  1.89596325  0.08491943
  0.18293709  1.7727413   0.6514795   0.96161842 -0.50265009 -0.69033495]
eval_val: 0.5034034627780741
index: 935
eval_count: 936
var_params: [ 0.2792949   1.77211672  2.17794307 -1.07301776  1.89593835  0.0835754
  0.18525828  1.77140738  0.6532425   0.

eval_count: 971
var_params: [ 0.31321478  1.78102486  2.18946589 -1.04120495  1.89600803  0.07433289
  0.17445342  1.76935718  0.64864921  0.95000927 -0.53322513 -0.66948015]
eval_val: 0.5032447824736332
index: 971
eval_count: 972
var_params: [ 0.31060649  1.78107041  2.19167702 -1.03977175  1.8990469   0.07710407
  0.17401936  1.76771611  0.64923001  0.94800953 -0.53716702 -0.6722399 ]
eval_val: 0.5032600549961935
index: 972
eval_count: 973
var_params: [ 0.31403546  1.78002175  2.19016145 -1.04197436  1.89620044  0.07259372
  0.17579901  1.76914038  0.64958887  0.94990812 -0.53548847 -0.66822796]
eval_val: 0.5032404060983835
index: 973
eval_count: 974
var_params: [ 0.31879853  1.78081634  2.19362608 -1.04230661  1.89233677  0.07098142
  0.17539138  1.76731449  0.65015738  0.94869879 -0.53419445 -0.66717189]
eval_val: 0.503227859583164
index: 974
eval_count: 975
var_params: [ 0.32105876  1.77704557  2.19230145 -1.04093023  1.88835793  0.07150622
  0.1757024   1.76722897  0.64815552  0.

## Storing the optimal parameters for grading

Once the training step of the vqc algorithm is done we obtain the optimal parameters for our specific variational form. For the grading function to be able to access these optimal parameters you will need to follow the steps below. 

* **Step 1**: Run the cell below with `print(repr(vqc.optimal_params))`. 
* **Step 2**: Copy the matrix of optimal parameters and store it in the variable `optimal_parameters` inside the function `return_optimal_params()` in the next cell. This will enable us to extract it while calculating the accuracy your the model during grading. Given below is a pictographical explanation of the same:  

<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/opt_params456b075.png" width="800">

In [34]:
print(repr(vqc.optimal_params))

array([ 0.33210605,  1.78792044,  2.19950557, -1.0299355 ,  1.89075252,
        0.0730189 ,  0.16818096,  1.76456468,  0.64982098,  0.94678557,
       -0.54047188, -0.65779748])


In [42]:
%%write_and_run optimal_params.py
# # the write_and_run function writes the content in this cell into the file "optimal_params.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
import numpy as np
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def return_optimal_params():
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    
    optimal_parameters = None
    optimal_parameters = [ 0.33210605,  1.78792044,  2.19950557, -1.0299355 ,  1.89075252,
        0.0730189 ,  0.16818096,  1.76456468,  0.64982098,  0.94678557,
       -0.54047188, -0.65779748]
    
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    return np.array(optimal_parameters)

## Submission

Before we go any further, check that you have the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` in the **same working directory as this notebook**. If you do not, then go back to the start and run the notebook making sure you have filled in the code where its required. When you run the cell below, all the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` are combined into one file named **"answer.py"**. Now your working directory will have four python (.py) files out of which **"answer.py"** is the submission file: 
* `answer.py` <- upload this file onto HackerEarth and click on "Submit and Evaluate"
* `feature_map.py`
* `variational_circuit.py`
* `optimal_params.py`

In [43]:
solution = ['feature_map.py','variational_circuit.py','optimal_params.py']
file = open("answer.py","w")
file.truncate(0)
for i in solution:    
    with open(i) as f:
        with open("answer.py", "a") as f1:
            for line in f:
                f1.write(line)
file.close()

## Grading Function

Given below is the grading function that we shall use to grade your submission with a test dataset that is of the same format as `challenge_dataset_4_9.csv`. You can use it to grade your submission by extracting a few points out of the `challenge_dataset_4_9.csv` to get a basic idea of how your model is performing. 

In [44]:
#imports required for the grading function 
from qiskit import *
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit.aqua.components.variational_forms import VariationalForm
import numpy as np

### Working of the grading function

The grading function `grade()` takes as **input**: 

* `test_data`: (`np.ndarray`) -- **no. of datapoints $\times$ dimension of data** : the datapoints against which we want to test our model. 


* `test_labels`: (`np.ndarray`) -- **no. of datapoints $\times$ 1** : A column vector with each entry either 0 or 1 as entries.


* `feature_map`: (`QuantumCircuit` or `FeatureMap`) -- A quantum feature map which is the output of `feature_map()` defined earlier.


* `variational_form`: (`QuantumCircuit` or `VariationalForm`) -- A variational form which is the output of `variational_circuit()` defined earlier.


* `optimal_params`: (`numpy.ndarray`) -- the optimal parameters obtained after running the VQC algorithm above. These are the values obtained when the function `return_optimal_params()` is run. 


* `find_circuit_cost` : (`bool`) -- Calculates the circuit cost if set to `True`. Circuit cost is calculated by converting the circuit to the basis gate set `\[ 'u3', 'cx'\]` and then applying the formula **cost = 1$\times$(no.of u3 gates) + 10$\times$(no.of cx gates)**.


* `verbose` : (`bool`) -- prints the result message if set to `True`.

And gives as **output**: 

* `model_accuracy` : (`numpy.float64`) -- percent accuracy of the model. 


* `circuit_cost`: (`int`) -- circuit cost as explained above.


* `ans`: (`tuple`) -- Output of the `VQC.predict()` method. 


* `result_msg`: (`str`) -- Result message which also outputs the error message in case of one.


* `unrolled_circuit`: (`QuantumCircuit` or `None`) -- the circuit obtained after unrolling the full VQC circuit and substituting the optimal parameters to the basis gate set `\[ 'u3', 'cx'\]`.

**Note:** if you look inside the `grade()` function in Section 2 you'll see that we have initialized a COBYLA optimizer though the prediction step will not require one. Similarily we have given a dataset to `training dataset`. Both of these are dummy variables. The reason for this is because these are not optional variables the `VQC` class instantiation.  

In [45]:
def grade(test_data, test_labels, feature_map, variational_form, optimal_params, find_circuit_cost=True, verbose=True):
    seed = 10598
    model_accuracy = None 
    circuit_cost=None 
    ans = None
    unrolled_circuit = None
    result_msg=''
    data_dim = np.array(test_data).shape[1]
    dataset_size = np.array(test_data).shape[0]
    dummy_training_dataset=training_input = {'A':np.ones((2,data_dim)), 'B':np.ones((2, data_dim))}
    
    # converting 4's to 0's and 9's to 1's for checking 
    test_labels_transformed = np.where(test_labels==4, 0., 1.)
    max_qubit_count = 6
    max_circuit_cost = 2000
    
    # Section 1
    if feature_map is None:
        result_msg += 'feature_map variable is None. Please submit a valid entry' if verbose else ''
    elif variational_form is None: 
        result_msg += 'variational_form variable is None. Please submit a valid entry' if verbose else ''
    elif optimal_params is None: 
        result_msg += 'optimal_params variable is None. Please submit a valid entry' if verbose else ''
    elif test_data is None: 
        result_msg += 'test_data variable is None. Please submit a valid entry' if verbose else ''
    elif test_labels is None: 
        result_msg += 'test_labels variable is None. Please submit a valid entry' if verbose else ''
    elif not isinstance(feature_map, (QuantumCircuit, FeatureMap)):
        result_msg += 'feature_map variable should be a QuantumCircuit or a FeatureMap not (%s)' % \
                      type(feature_map) if verbose else ''
    elif not isinstance(variational_form, (QuantumCircuit, VariationalForm)):
        result_msg += 'variational_form variable should be a QuantumCircuit or a VariationalForm not (%s)' % \
                      type(variational_form) if verbose else ''
    elif not isinstance(test_data, np.ndarray):
        result_msg += 'test_data variable should be a numpy.ndarray not (%s)' % \
                      type(test_data) if verbose else ''
    elif not isinstance(test_labels, np.ndarray):
        result_msg += 'test_labels variable should be a numpy.ndarray not (%s)' % \
                      type(test_labels) if verbose else ''
    elif not isinstance(optimal_params, np.ndarray):
        result_msg += 'optimal_params variable should be a numpy.ndarray not (%s)' % \
                      type(optimal_params) if verbose else ''
    elif not dataset_size == test_labels_transformed.shape[0]:
        result_msg += 'Dataset size and label array size must be equal'
    # Section 2
    else:
        
        # setting up COBYLA optimizer as a dummy optimizer
        from qiskit.aqua.components.optimizers import COBYLA
        dummy_optimizer = COBYLA()

        # setting up the backend and creating a quantum instance
        backend = Aer.get_backend('qasm_simulator')
        backend_options = {"method": "statevector"}
        quantum_instance = QuantumInstance(backend, 
                                           shots=2000, 
                                           seed_simulator=seed, 
                                           seed_transpiler=seed, 
                                           backend_options=backend_options)

        # creating a VQC instance and running the VQC.predict method to get the accuracy of the model 
        vqc = VQC(optimizer=dummy_optimizer, 
                  feature_map=feature_map, 
                  var_form=variational_form, 
                  training_dataset=dummy_training_dataset)
        
        from qiskit.transpiler import PassManager
        from qiskit.transpiler.passes import Unroller
        pass_ = Unroller(['u3', 'cx'])
        pm = PassManager(pass_)
        # construct circuit with first datapoint
        circuit = vqc.construct_circuit(data[0], optimal_params)
        unrolled_circuit = pm.run(circuit)
        gates = unrolled_circuit.count_ops()
        if 'u3' in gates: 
            circuit_cost = gates['u3']
        if 'cx' in gates: 
            circuit_cost+= 10*gates['cx']
        
        if circuit.num_qubits > max_qubit_count:
            result_msg += 'Your quantum circuit is using more than 6 qubits. Reduce the number of qubits used and try again.'
        elif circuit_cost > max_circuit_cost:
            result_msg += 'The cost of your circuit is exceeding the maximum accpetable cost of 2000. Reduce the circuit cost and try again.'
        else: 
            
            ans = vqc.predict(test_data, quantum_instance=quantum_instance, params=np.array(optimal_params))
            model_accuracy = np.sum(np.equal(test_labels_transformed, ans[1]))/len(ans[1])

            result_msg += 'Accuracy of the model is {}'.format(model_accuracy) if verbose else ''
            result_msg += ' and circuit cost is {}'.format(circuit_cost) if verbose else ''
            
    return model_accuracy, circuit_cost, ans, result_msg, unrolled_circuit

## Process of grading using a dummy grading dataset

Let us create a dummy grading dataset with features and labels `grading_features` and `grading_labels` created from the last 2000 datapoints from `data_features` and `data_labels`so that we can a rough estimate of our accuaracy. It must be noted that this may not be a balanced dataset, i.e, may not have equal number of `4`'s and `9`'s and is not best practice. This is only given for the purpose of the demo of `grade()` function. In the final scoring done on HackerEarth, the testing dataset used will have a balanced number of class labels `4` and `9`.

In [46]:
grading_dataset_size=2000    # this value is not per digit but in total
grading_features = data_features[-grading_dataset_size:]
grading_labels = data_labels[-grading_dataset_size:]

In [47]:
start = time.process_time()

accuracy, circuit_cost, ans, result_msg, full_circuit  =  grade(test_data=grading_features, 
                                                                test_labels=grading_labels, 
                                                                feature_map=feature_map(), 
                                                                variational_form=variational_circuit(), 
                                                                optimal_params=return_optimal_params())

print("time taken: {} seconds".format(time.process_time() - start))
print(result_msg)

time taken: 22.03125 seconds
Accuracy of the model is 0.729 and circuit cost is 447


You can also check your **accuracy**, **circuit_cost** and **full_circuit** which is the result of combining the feature map and variational circuit and unrolling into the basis \['u3', 'cx'\].

In [48]:
print("Accuracy of the model: {}".format(accuracy))
print("Circuit Cost: {}".format(circuit_cost))
print("The complete unrolled circuit: ")
full_circuit.draw()

Accuracy of the model: 0.729
Circuit Cost: 447
The complete unrolled circuit: 


┌───────────────┐  ┌───────────┐                               »
q_0: ┤ U3(pi/2,0,pi) ├──┤ U3(0,0,8) ├─────■─────────────────■────■──»
     ├───────────────┤┌─┴───────────┴──┐┌─┴─┐┌───────────┐┌─┴─┐  │  »
q_1: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,1.6456) ├┤ X ├┤ U3(0,0,0) ├┤ X ├──┼──»
     ├───────────────┤├────────────────┤└───┘└───────────┘└───┘┌─┴─┐»
q_2: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,1.8033) ├───────────────────────┤ X ├»
     └───────────────┘└────────────────┘                       └───┘»
c_0: ═══════════════════════════════════════════════════════════════»
                                                                    »
c_1: ═══════════════════════════════════════════════════════════════»
                                                                    »
c_2: ═══════════════════════════════════════════════════════════════»
                                                                    »
«                       ┌───────────────┐┌───────────┐                      »
«q_0: ───────────────■──┤ U3(pi/2,0,pi) ├┤ U3(0,0,8) ├──────────────────────»
«                    │  └───────────────┘└───────────┘     ┌───────────────┐»
«q_1: ───────────────┼──────────■───────────────────────■──┤ U3(pi/2,0,pi) ├»
«     ┌───────────┐┌─┴─┐      ┌─┴─┐      ┌───────────┐┌─┴─┐├───────────────┤»
«q_2: ┤ U3(0,0,0) ├┤ X ├──────┤ X ├──────┤ U3(0,0,0) ├┤ X ├┤ U3(pi/2,0,pi) ├»
«     └───────────┘└───┘      └───┘      └───────────┘└───┘└───────────────┘»
«c_0: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«c_1: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«c_2: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«                                                                     »
«q_0: ────────────────────■─────────────────■────■─────────────────■──»
«     ┌────────────────┐┌─┴─┐┌───────────┐┌─┴─┐  │                 │  »
«q_1: ┤ U3(0,0,1.6456) ├┤ X ├┤ U3(0,0,0) ├┤ X ├──┼─────────────────┼──»
«     ├────────────────┤└───┘└───────────┘└───┘┌─┴─┐┌───────────┐┌─┴─┐»
«q_2: ┤ U3(0,0,1.8033) ├───────────────────────┤ X ├┤ U3(0,0,0) ├┤ X ├»
«     └────────────────┘                       └───┘└───────────┘└───┘»
«c_0: ════════════════════════════════════════════════════════════════»
«                                                                     »
«c_1: ════════════════════════════════════════════════════════════════»
«                                                                     »
«c_2: ════════════════════════════════════════════════════════════════»
«                                                                     »
«     ┌───────────────┐┌───────────┐                                        »
«q_0: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,8) ├────────────────────────────────────────»
«     └───────────────┘└───────────┘     ┌───────────────┐┌────────────────┐»
«q_1: ────────■───────────────────────■──┤ U3(pi/2,0,pi) ├┤ U3(0,0,1.6456) ├»
«           ┌─┴─┐      ┌───────────┐┌─┴─┐├───────────────┤├────────────────┤»
«q_2: ──────┤ X ├──────┤ U3(0,0,0) ├┤ X ├┤ U3(pi/2,0,pi) ├┤ U3(0,0,1.8033) ├»
«           └───┘      └───────────┘└───┘└───────────────┘└────────────────┘»
«c_0: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«c_1: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«c_2: ══════════════════════════════════════════════════════════════════════»
«                                                                           »
«                                                   ┌───────────────┐»
«q_0: ──■─────────────────■────■─────────────────■──┤ U3(pi/